In [1]:
using LazySets, BenchmarkTools

## baseline

In [5]:
P1 = VPolytope([[0.0, 0.0], [2.0, 1.0], [1.0, 2.0]]);
P2 = VPolytope([rand(2) for _ in 1:100]);
P3 = VPolytope([rand(2) for _ in 1:1000]);

In [15]:
@btime box_approximation($P1)
@btime box_approximation($P2)
@btime box_approximation($P3)

  155.359 ns (3 allocations: 224 bytes)
  2.414 μs (3 allocations: 224 bytes)
  23.138 μs (3 allocations: 224 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.5001524552357717, 0.5006015326369077], [0.49933757531871903, 0.4993320190149356])

## version 1

In [4]:
function ba1(P::Union{VPolytope, VPolygon})
    n = dim(P)
    vlist = vertices_list(P)
    low = copy(vlist[1])
    high = copy(vlist[1])
    @inbounds for v in @view vlist[2:length(vlist)]
        for i in 1:n
            if v[i] > high[i]
                high[i] = v[i]
            elseif v[i] < low[i]
                low[i] = v[i]
            end
        end
    end
    return Hyperrectangle(low=low, high=high)
end

ba1 (generic function with 1 method)

In [9]:
@btime ba1($P1)
@btime ba1($P2)
@btime ba1($P3)

  118.679 ns (4 allocations: 384 bytes)
  319.459 ns (4 allocations: 384 bytes)
  2.465 μs (4 allocations: 384 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.5001524552357717, 0.5006015326369077], [0.49933757531871903, 0.4993320190149356])

## version 2

In [10]:
function ba2(P::Union{VPolytope, VPolygon})
    n = dim(P)
    vlist = vertices_list(P)
    v1 = vlist[1]
    center = similar(v1)
    radius = similar(v1)
    
    @inbounds begin
    for i in 1:n
        high_i = v1[i]
        low_i = v1[i]
        for v in vlist
            if v[i] > high_i
                high_i = v[i]
            elseif v[i] < low_i
                low_i = v[i]
            end
        end
        center[i] = (high_i + low_i)/2
        radius[i] = (high_i - low_i)/2
    end
    end
    return Hyperrectangle(center, radius)
end

ba2 (generic function with 1 method)

In [11]:
@btime ba2($P1)
@btime ba2($P2)
@btime ba2($P3)

  60.412 ns (2 allocations: 192 bytes)
  227.487 ns (2 allocations: 192 bytes)
  2.067 μs (2 allocations: 192 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.5001524552357717, 0.5006015326369077], [0.49933757531871903, 0.4993320190149356])

## using static arrays

In [16]:
using StaticArrays

In [23]:
P1s = VPolytope([SA[0.0, 0.0], SA[2.0, 1.0], SA[1.0, 2.0]]);
P2s = VPolytope([SVector{2}(v) for v in P2.vertices])
P3s = VPolytope([SVector{2}(v) for v in P3.vertices]);

In [25]:
P1s

VPolytope{Float64,SArray{Tuple{2},Float64,1,2}}(SArray{Tuple{2},Float64,1,2}[[0.0, 0.0], [2.0, 1.0], [1.0, 2.0]])

In [27]:
@btime box_approximation($P1s)
@btime box_approximation($P2s)
@btime box_approximation($P3s)

  92.226 ns (3 allocations: 224 bytes)
  566.342 ns (3 allocations: 224 bytes)
  4.808 μs (3 allocations: 224 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.5001524552357717, 0.5006015326369077], [0.49933757531871903, 0.4993320190149356])

In [30]:
@btime ba1($P1s)
@btime ba1($P2s)
@btime ba1($P3s)

ErrorException: setindex!(::SArray{Tuple{2},Float64,1,2}, value, ::Int) is not defined.

In [29]:
@btime ba2($P1s)
@btime ba2($P2s)
@btime ba2($P3s)

  25.580 ns (2 allocations: 64 bytes)
  235.806 ns (2 allocations: 64 bytes)
  2.094 μs (2 allocations: 64 bytes)


Hyperrectangle{Float64,MArray{Tuple{2},Float64,1,2},MArray{Tuple{2},Float64,1,2}}([0.5001524552357717, 0.5006015326369077], [0.49933757531871903, 0.4993320190149356])

(we should make it return an SArray though)

---

## higher dim

In [33]:
n = 3
Pn = VPolytope([rand(n) for _ in 1:50]);
Pns = VPolytope([SVector{n}(v) for v in Pn.vertices]);

@btime box_approximation($Pn)
@btime ba1($Pn)
@btime ba2($Pn)

  2.181 μs (3 allocations: 256 bytes)
  249.644 ns (4 allocations: 448 bytes)
  173.296 ns (2 allocations: 224 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.5168276314349014, 0.4907079638498182, 0.4999135699048741], [0.4788282555560608, 0.46966803773451127, 0.4885036871807852])

In [34]:
n = 5
Pn = VPolytope([rand(n) for _ in 1:100]);
Pns = VPolytope([SVector{n}(v) for v in Pn.vertices]);

@btime box_approximation($Pn)
@btime ba1($Pn)
@btime ba2($Pn)

  9.000 μs (3 allocations: 288 bytes)
  572.443 ns (4 allocations: 512 bytes)
  533.323 ns (2 allocations: 256 bytes)


Hyperrectangle{Float64,Array{Float64,1},Array{Float64,1}}([0.48462291158877624, 0.5020136795798725, 0.4967681788269117, 0.5017653900751188, 0.49769085009698255], [0.48102515713246297, 0.4915865772532847, 0.489882239971385, 0.49519555569823426, 0.4831677513810295])

In [38]:
n = 10
Pn = VPolytope([rand(n) for _ in 1:2000]);
Pns = VPolytope([SVector{n}(v) for v in Pn.vertices]);

@btime box_approximation($Pn)
@btime ba1($Pn)
@btime ba2($Pn)

@btime box_approximation($Pns)
#@btime ba1($Pn)
@btime ba2($Pns)

  547.953 μs (3 allocations: 352 bytes)
  16.620 μs (4 allocations: 640 bytes)
  33.969 μs (2 allocations: 320 bytes)
  171.852 μs (3 allocations: 352 bytes)
  32.785 μs (2 allocations: 192 bytes)


Hyperrectangle{Float64,MArray{Tuple{10},Float64,1,10},MArray{Tuple{10},Float64,1,10}}([0.5001066046266438, 0.5001489974069424, 0.4999021012172842, 0.49948876478262516, 0.4998057393859223, 0.4999505607691883, 0.5006176904611697, 0.5001747187983653, 0.49997998417084444, 0.4999477078899164], [0.49983367441612647, 0.49983824292841694, 0.4997680183026787, 0.49936112496504537, 0.4996588354216087, 0.49972763186387104, 0.49923789680635045, 0.49980277694838093, 0.49937012316215756, 0.4997960261956038])